# Download HaGRID data set from kaggle

In [3]:
# Install dependencies as needed:
!pip install kagglehub[pandas-datasets]

In [ ]:
pip install kaggle

## Download dataset from Kaggle using kagglehub

Use this data set because:
* Size and diversity: >550K images, 18 static gestures, different conditions (lighting, background, poses).
* Diversity of people: many different hands, ages, genders - less risk of overfitting.
* Realistic conditions: real-life images with different backgrounds, viewing angles
* Ready-made gesture classes: (✊, ✋, ✌️, etc.) can be easily expanded for our custom idea


In [1]:
import kagglehub

def download_hagrid(output_dir="data/raw/hagrid"):
    # Download latest version
    path = kagglehub.dataset_download("kapitanov/hagrid")

    print("Path to dataset files:", path)

C:\Users\flees\Desktop\F25\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 338M/338M [09:34<00:00, 616kB/s] 

Extracting files...


Path to dataset files: C:\Users\flees\.cache\kagglehub\datasets\kapitanov\hagrid\versions\3
